In [ ]:
# /dbfs/FileStore/json/edp_bronze_json
#dbutils.widgets.text("directory", "")
folder_path = dbutils.widgets.get("directory")
output_path = dbutils.widgets.get("DDL_file_path")
source_path_dev = dbutils.widgets.get("source_path_dev")

In [ ]:
import os,json
# dir_path = '/dbfs/FileStore/Json_Aug23'  #'/dbfs/FileStore/Json'
ss_missing = []
for file in os.listdir(folder_path):
    filename = os.path.join(folder_path, file)
    with open(filename, 'r') as data:
        item = json.load(data)[0]
        par_name = item["source_details"]["source_table"].lower()
        data_path = f"""{source_path_dev}{par_name}"""
        # data_path = f"""/mnt/landing/Blckstrw/{par_name}"""
        try:
            df = spark.read.parquet(data_path)
            schema = df.schema
            schema_str = ", ".join([f"{field.name}:{field.dataType.simpleString()}" for field in schema])
        # print(schema_str)
            with open(f"""{output_path}{item["source_details"]["source_table"].lower()}.ddl""", "w") as file:
                file.write(schema_str)
                print(f">> DDL created for {filename}")
        except:
            ss_missing.append(data_path)
            continue


print("/n missing: ", ss_missing)